## To-do List
- Import necessaries libraries
- Put our API from google
- Connect to map_client, give query and distance
- Run the script
- in the 'results' only get the neccessary columns
- Run the Script
- Get the results and append it as a dataframe
- Still learning to filll out the places id from the result we append, then use the list of place id to filter it from running again (still failed)
- drop duplicates if any
- import to CSV to be visualized


In [1]:
import googlemaps
import pandas as pd
import pickle
import time
from pandas import json_normalize
import requests

In [2]:
api_key = 'insert your apis'
map_client = googlemaps.Client(api_key)
locations = [] # add your langtitude, longtitude
# locations = [(-8.68525, 115.20442),(-8.20001, 113.64362)
#              ,(-7.96529, 112.63065),(-7.2895, 112.71629),(-7.82088, 111.98943),(-7.56964, 110.8071),(-7.78337, 110.3836), (-7.13973, 110.42268),(-7.46906, 109.28698)]
# Bali > Jember > Malang > Surabaya > Kediri > Surakarta > Yogyakarta > Semarang > Purwokerto

search_str='mixue OR Mixue OR MIXUE'
distance = 50000 #in metres


In [6]:
# Replace YOUR_API_KEY with your actual API key
gmaps = googlemaps.Client(key='insert your apis')

# Set the type of place to Mixue
place_type = 'Mixue'

# Set the list of locations to search
locations = [(-5.426112539367408, 105.25786398164121),(-2.9555158764720386, 104.76495584674291),(-1.5820630973021552, 103.60401718195902),(0.5185930176320304, 101.42421553886894),(3.5930341821817064, 98.67437858620407),(1.0810415713289183, 104.00798092002987),(-1.619477308647127, 103.60010883810612),(0.5560670951166239, 123.04924359319672),(-5.152025364571885, 119.41903718065586),(-4.447843694480461, 120.3097566625435),(-0.9564194116204323, 119.88662739556123),(-0.9583603765210935, 122.78525219960646)]

# Create an empty list to store the results
results = []

# Loop through the list of locations
for location in locations:
  # Initialize the page token
  page_token = None
  # Set a flag to check if there are more pages
  more_pages = True
  # Loop through the pages of results
  while more_pages:
    # Search for Mixue places in the location
    places = gmaps.places(query=place_type, location=location, page_token=page_token,radius=50000)
    # Loop through the search results
    for place in places['results']:
      # Get the detailed information for each place
      branch = gmaps.place(place_id=place['place_id'])['result']
      results.append(branch)
    # Update the page token and check if there are more pages
    page_token = places.get('next_page_token')
    more_pages = page_token is not None


In [7]:
len(results)

720

In [8]:

results_empty = []

for item in results:
    result = {}
    result['business_status'] = item['business_status']
    result['delivery'] = item.get('delivery', "N/A")
    result['dine_in'] = item.get('dine_in', "N/A")
    result['formatted_address'] = item['formatted_address']
    for component in item['address_components']:
        if 'administrative_area_level_1' in component['types']:
            result['location'] = component['long_name']
        elif 'administrative_area_level_2' in component['types']:
            result['location2'] = component['long_name']
        elif 'administrative_area_level_3' in component['types']:
            result['location3'] = component['long_name']
    result['formatted_phone_number'] = item.get('formatted_phone_number', "N/A")
    result['lat'] = item['geometry']['location']['lat']
    result['lng'] = item['geometry']['location']['lng']
    result['name'] = item['name']
    result['place_id'] = item['place_id']
    result['price_level'] = item.get('price_level', "N/A")
    result['rating'] = item.get('rating', "N/A")
    result['url'] = item['url']
    # Make a request to the Place Details endpoint to retrieve the user_ratings_total field
    place_details_url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={item['place_id']}&fields=user_ratings_total&key={api_key}"
    place_details_response = requests.get(place_details_url)
    place_details_data = place_details_response.json()
    # Add the user_ratings_total field to the result dictionary
    result['user_ratings_total'] = place_details_data['result'].get('user_ratings_total', "N/A")
    results_empty.append(result)

df = pd.DataFrame(results_empty)


In [28]:
df.tail(5)

,business_status,delivery,dine_in,formatted_address,location3,location2,location,formatted_phone_number,lat,lng,name,place_id,price_level,rating,url,user_ratings_total
715,OPERATIONAL,True,True,"Jl. Ciremai Raya No.D15, Larangan, Kec. Harjam...",Kecamatan Harjamukti,Kota Cirebon,Jawa Barat,0819-0304-8999,-6.742784,108.563423,Mixue Ciremai,ChIJdWTX6cUdby4R82RVSxirwHA,2,4.7,https://maps.google.com/?cid=8124681848607827187,280
716,OPERATIONAL,True,True,"Jl. Hasanudin No.A47, Panggung Lor, Kec. Semar...",Kecamatan Semarang Utara,Kota Semarang,Jawa Tengah,N/A,-6.960182,110.410648,Mixue Tanah Mas,ChIJKfu_KCX1cC4Rn3ssF4SE2m8,N/A,4.7,https://maps.google.com/?cid=8059900185992264607,188
717,OPERATIONAL,True,True,"Jl. Raya Klp. Hybrida Blok PD14, RT.4/RW.18, P...",Kelapa Gading,North Jakarta City,Jakarta,0895-4013-68699,-6.148602,106.911145,Mixue Kelapa Gading Hybrida,ChIJZyLV65L1aS4RXz61rIvDih8,N/A,4.5,https://maps.google.com/?cid=2272843966627266143,153
718,OPERATIONAL,True,True,"Jl. Kaliurang No.9,3, Gondangan, Sinduharjo, K...",Kecamatan Ngaglik,Kabupaten Sleman,Daerah Istimewa Yogyakarta,N/A,-7.727047,110.399304,Mixue Jakal,ChIJb2n89IRZei4R9zxsqAM9OEQ,N/A,4.6,https://maps.google.com/?cid=4915746079194758391,566
719,OPERATIONAL,True,True,"Jl. Terusan Pasirkoja No.112, Jamika, Kec. Boj...",Kecamatan Bojongloa Kaler,Kota Bandung,Jawa Barat,0819-0110-0441,-6.925624,107.589274,Mixue Pasir Koja,ChIJs5V5H4TnaC4RdrEggq908CY,N/A,4.5,https://maps.google.com/?cid=2805870865003098486,191


In [10]:
print(df['place_id'].duplicated().sum())
print(df.shape)

433
(720, 16)


In [11]:
df_no_duplicate = df.drop_duplicates()

In [35]:
print(df['place_id'].duplicated().sum())
print(df_no_duplicate.shape)
#after dropping duplicate

433
(287, 16)


In [13]:
df_no_duplicate.to_csv('saving-csv.csv')